In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import shutil
import warnings
import csv
import yaml
import json
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Any, TypedDict

from megadetector.detection.run_detector import load_detector, model_string_to_model_version
from megadetector.detection.run_detector_batch import process_images, write_results_to_file

from os import PathLike
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split


In [2]:
from ba_dev.dataloader import MammaliaData

In [3]:
def load_path_config(path_to_config):
    with open(path_to_config, 'r') as f:
        path_config = yaml.safe_load(f)
    return {k: Path(v) for k, v in path_config.items()}

paths = load_path_config('../path_config.yml')


### Running Tests

In [9]:
paths['testset']

PosixPath('/Volumes/ExSSD/UserData/GoogleDrive/01_files/001_Studium/001 Semester/06 FS25/09_BA/data/test_set')

In [ ]:
path_to_testset = Path('/cfs/earth/scratch/kraftjul/BA/data/test_set')
output_path = Path('/cfs/earth/scratch/kraftjul/BA/output')


In [4]:
path_to_dataset = paths['dataset']
path_labelfiles = paths['testset']
path_to_detector_output = path_labelfiles / 'md_out'
detector_model = 'mdv5a'
mode = 'init'

dataset = MammaliaData(
    path_to_dataset = path_to_dataset,
    path_labelfiles = path_labelfiles,
    path_to_detector_output = path_to_detector_output,
    detector_model = detector_model,
    mode = mode,
    )

Bypassing download of already-downloaded file md_v5a.0.0.pt
Model v5a.0.0 available at /var/folders/t0/5px6w93n5rvdnqbxvt4n6qf00000gn/T/megadetector_models/md_v5a.0.0.pt


Fusing layers... 
Fusing layers... 


Loading PT detector with compatibility mode classic


Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
Model summary: 733 layers, 140054656 parameters, 0 gradients, 208.8 GFLOPs
/Users/jk/MyPythonPackages/BA_package/ba_dev/dataloader.py:219: UserWarning: With the detection confidence of 0.25,
25 sequences of the full set had no detections and will be excluded.
Excluded sequences: [4003328, 4000256, 4000898, 4000523, 6000145, 6000019, 6000276, 6000924, 4001442, 2000045] ...
  warnings.warn(


In [ ]:
dataset[0]



Methode changed


In [19]:
files = path_to_labelfiles.glob('*')

metadata = pd.DataFrame()
categories_to_drop = ['other', 'glis_glis']

for file in files:
    metadata = pd.concat([metadata, pd.read_csv(file)], ignore_index=True)
    metadata = metadata.dropna(subset=['label2'])
    metadata = metadata[~metadata['label2'].isin(categories_to_drop)]


In [20]:
metadata

,SerialNumber,seq_nr,Directory,DateTime_start,DateTime_end,duration_seconds,first_file,last_file,n_files,all_files,label,duplicate_label,label2
0,H,2,sessions/session_01/H_2,2019-08-31T01:47:33Z,2019-08-31T01:47:45Z,12.0,RCNX0005.JPG,RCNX0011.JPG,6,"RCNX0005.JPG,RCNX0006.JPG,RCNX0007.JPG,RCNX000...",apodemus_sp,0.0,apodemus_sp
1,H,3,sessions/session_01/H_2,2019-09-16T11:40:27Z,2019-09-16T11:40:39Z,12.0,RCNX0013.JPG,RCNX0019.JPG,6,"RCNX0013.JPG,RCNX0014.JPG,RCNX0015.JPG,RCNX001...",mustela_erminea,0.0,mustela_erminea
2,H,4,sessions/session_01/H_2,2019-09-16T11:41:12Z,2019-09-16T11:41:14Z,2.0,RCNX0021.JPG,RCNX0023.JPG,3,"RCNX0021.JPG,RCNX0022.JPG,RCNX0023.JPG",mustela_erminea,0.0,mustela_erminea
3,H,5,sessions/session_01/H_2,2019-09-16T11:46:10Z,2019-09-16T11:46:11Z,1.0,RCNX0025.JPG,RCNX0027.JPG,3,"RCNX0025.JPG,RCNX0026.JPG,RCNX0027.JPG",mustela_erminea,0.0,mustela_erminea
4,H,6,sessions/session_01/H_2,2019-09-16T11:46:46Z,2019-09-16T11:46:47Z,1.0,RCNX0029.JPG,RCNX0031.JPG,3,"RCNX0029.JPG,RCNX0030.JPG,RCNX0031.JPG",mustela_erminea,0.0,mustela_erminea
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22135,H550HF08161376,261,sessions/session_02/H550HF08161376_2,2019-08-21T12:55:01Z,2019-08-21T12:56:25Z,84.0,IMG_4312_2.JPG,IMG_4356_2.JPG,45,"IMG_4312_2.JPG,IMG_4313_2.JPG,IMG_4314_2.JPG,I...",microtus_sp,NaN,cricetidae
22136,H550HF08161376,262,sessions/session_02/H550HF08161376_2,2019-08-21T12:57:24Z,2019-08-21T12:58:17Z,53.0,IMG_4357_2.JPG,IMG_4401_2.JPG,45,"IMG_4357_2.JPG,IMG_4358_2.JPG,IMG_4359_2.JPG,I...",microtus_sp,NaN,cricetidae
22137,H550HF08161376,263,sessions/session_02/H550HF08161376_2,2019-08-21T17:25:17Z,2019-08-21T17:27:01Z,104.0,IMG_4402_2.JPG,IMG_4479_2.JPG,78,"IMG_4402_2.JPG,IMG_4403_2.JPG,IMG_4404_2.JPG,I...",microtus_sp,NaN,cricetidae
22138,H550HF08161376,264,sessions/session_02/H550HF08161376_2,2019-08-21T17:27:45Z,2019-08-21T17:28:19Z,34.0,IMG_4480_2.JPG,IMG_4509_2.JPG,30,"IMG_4480_2.JPG,IMG_4481_2.JPG,IMG_4482_2.JPG,I...",microtus_sp,NaN,cricetidae


### Sampling dataset

In [4]:
paths['labels']

PosixPath('/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels')

In [12]:
import chardet

path = paths['labels']

for path in paths['labels'].glob('*'):
    print(path)
    with open(path, 'rb') as f:
        result = chardet.detect(f.read())
        print(f"File: {path}, Encoding: {result['encoding']}")


/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_01_labels.csv
File: /Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_01_labels.csv, Encoding: ascii
/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/.DS_Store
File: /Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/.DS_Store, Encoding: Windows-1252
/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_04_labels.csv
File: /Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_04_labels.csv, Encoding: ascii
/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_03_labels.csv
File: /Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_03_labels.csv, Encoding: ascii
/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_06_labels.csv
File: /Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_06_labels.csv, Encoding: ascii
/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_05_labels.csv
File: /Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_05_labels.csv, Encodin

In [23]:
# Paths
path_to_labelfiles = paths['labels']
dataset_root = paths['dataset']
target_dir = paths['testset']
output_metadata_csv = target_dir / "metadata_sample_set.csv"


files = path_to_labelfiles.glob('*.csv')
metadata = pd.DataFrame()
categories_to_drop = ['other', 'glis_glis']

for file in files:
    metadata = pd.concat([metadata, pd.read_csv(file)], ignore_index=True)
    metadata = metadata.dropna(subset=['label2'])
    metadata = metadata[~metadata['label2'].isin(categories_to_drop)]


metadata_filtered = metadata[metadata['n_files']<60]

metadata_sampled = metadata_filtered.groupby("label2", group_keys=False).sample(n=40, random_state=42)

metadata_sampled.to_csv(output_metadata_csv, index=False)


In [20]:
paths['labels']

PosixPath('/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels')

In [22]:
files = paths['labels'].glob('*.csv')
for file in files:
    print(file)


/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_01_labels.csv
/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_04_labels.csv
/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_03_labels.csv
/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_06_labels.csv
/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_05_labels.csv
/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_07_labels.csv
/Volumes/ExSSD/UserData/BA_Data/dataset/info/labels/session_02_labels.csv


In [ ]:
metadata



""


In [7]:
# Paths
path_to_labelfiles = Path("/cfs/earth/scratch/iunr/shared/iunr-mammaliabox/dataset/info/labels")
dataset_root = Path("/cfs/earth/scratch/iunr/shared/iunr-mammaliabox/dataset")
target_dir = Path("/cfs/earth/scratch/kraftjul/BA/data/test_set_large")
output_metadata_csv = target_dir / "metadata_larger_sample_set.csv"


# Load metadata
metadata = dataset.reading_all_metadata(
    list_of_files=dataset.getting_all_files_of_type(path_to_labelfiles, file_type='.csv'),
    categories_to_drop=[]
)

In [9]:
metadata

,session,SerialNumber,seq_nr,seq_id,Directory,DateTime_start,DateTime_end,duration_seconds,first_file,last_file,n_files,all_files,label,duplicate_label,label2
0,3,H550HF07158839,1,3000000,sessions/session_03/28,2020-03-29T20:57:38Z,2020-03-29T20:57:59Z,21.0,IMG_0001.JPG,IMG_0024.JPG,24,"IMG_0001.JPG,IMG_0002.JPG,IMG_0003.JPG,IMG_000...",apodemus_sp,NaN,apodemus_sp
1,3,H550HF07158878,1,3000003,sessions/session_03/42,2020-03-30T08:50:52Z,2020-03-30T08:50:54Z,2.0,IMG_0001.JPG,IMG_0003.JPG,3,"IMG_0001.JPG,IMG_0002.JPG,IMG_0003.JPG",mustela_erminea,NaN,mustela_erminea
2,3,H550HF07158878,2,3000004,sessions/session_03/42,2020-04-16T01:20:36Z,2020-04-16T01:20:41Z,5.0,IMG_0004.JPG,IMG_0009.JPG,6,"IMG_0004.JPG,IMG_0005.JPG,IMG_0006.JPG,IMG_000...",apodemus_sp,NaN,apodemus_sp
3,3,H550HF07158933,1,3000005,sessions/session_03/7,2020-04-11T14:05:39Z,2020-04-11T14:06:29Z,50.0,IMG_0001.JPG,IMG_0024.JPG,24,"IMG_0001.JPG,IMG_0002.JPG,IMG_0003.JPG,IMG_000...",apodemus_sp,NaN,apodemus_sp
4,3,H550HF07158933,2,3000006,sessions/session_03/7,2020-04-11T14:24:35Z,2020-04-11T14:24:44Z,9.0,IMG_0025.JPG,IMG_0031.JPG,7,"IMG_0025.JPG,IMG_0026.JPG,IMG_0027.JPG,IMG_002...",apodemus_sp,NaN,apodemus_sp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23101,4,H550HF08161368,235,4018684,sessions/session_04/W7-WK06,2020-07-17T23:42:26Z,2020-07-17T23:42:36Z,10.0,IMG_4228.JPG,IMG_4236.JPG,9,"IMG_4228.JPG,IMG_4229.JPG,IMG_4230.JPG,IMG_423...",apodemus_sp,False,apodemus_sp
23102,4,H550HF08161368,236,4018685,sessions/session_04/W7-WK06,2020-07-17T23:45:10Z,2020-07-17T23:45:34Z,24.0,IMG_4237.JPG,IMG_4248.JPG,12,"IMG_4237.JPG,IMG_4238.JPG,IMG_4239.JPG,IMG_424...",apodemus_sp,False,apodemus_sp
23103,4,H550HF08161368,237,4018686,sessions/session_04/W7-WK06,2020-07-18T01:56:54Z,2020-07-18T01:57:18Z,24.0,IMG_4249.JPG,IMG_4263.JPG,15,"IMG_4249.JPG,IMG_4250.JPG,IMG_4251.JPG,IMG_425...",apodemus_sp,False,apodemus_sp
23104,4,H550HF08161368,238,4018687,sessions/session_04/W7-WK06,2020-07-18T02:06:30Z,2020-07-18T02:06:42Z,12.0,IMG_4264.JPG,IMG_4269.JPG,6,"IMG_4264.JPG,IMG_4265.JPG,IMG_4266.JPG,IMG_426...",myodes_glareolus,False,cricetidae
